In [1]:
import json

#### Read Dataset & Entities

In [2]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [3]:
test_source_path = '/home/ml/cadencao/XSum/fairseq_files/train.source'
test_target_path = '/home/ml/cadencao/XSum/fairseq_files/train.target'

xsum_source = read_lines(test_source_path)
xsum_target = read_lines(test_target_path)

In [4]:
xsum_ents = json.load(open('xsum_train_ents.json', 'r'))

In [5]:
assert len(xsum_source) == len(xsum_target) == len(xsum_ents)

#### Build Clean Dataset

In [6]:
from tqdm import tqdm

In [7]:
train_src_clean, train_tgt_clean = [], []

for i in tqdm(range(len(xsum_source))):
#     src_ents = [xsum_source[i][e['start']:e['end']].lower() for e in xsum_ents[i]['src ents']]
    tgt_ents = [xsum_target[i][e['start']: e['end']].lower() for e in xsum_ents[i]['ents']]
    
    entiled = True
    for te in tgt_ents:
        if te not in xsum_source[i].lower():
            entiled = False
    
    if entiled:
        train_src_clean.append(xsum_source[i])
        train_tgt_clean.append(xsum_target[i])

100%|██████████| 203575/203575 [00:03<00:00, 60962.91it/s]


In [8]:
len(train_src_clean)

54019

In [9]:
train_tgt_clean[0]

'Fears that a nearby dam might collapse are reportedly causing panic and confusion in the Syrian city of Raqqa, a stronghold of the so-called Islamic State (IS) group.'

In [10]:
train_src_clean[0]

"These are external links and will open in a new window. IS media channels say US-led airstrikes weakened the Tabqa dam, but US-backed fighters denied hitting the facility. Some reports say civilians have begun to flee, while others say they are being reassured they can stay. US-backed Syrian rebels are trying to capture Raqqa from IS. On Wednesday, the Pentagon said members of the Syrian Democratic Forces (SDF) alliance had been airlifted to positions behind IS lines close to the Tabqa dam, which is about 40km (25 miles) upstream of the city on the Euphrates river. However, the BBC is not able to independently confirm the condition of the IS-held dam - which provides power to the region - nor the orders received by Raqqa's civilians. Earlier the anti-IS activist group, Raqqa is Being Slaughtered Silently, said people had started to flee the city - but that militants had been using loudspeakers to tell them that the dam was safe and that they should not leave. That comes despite IS cha

In [11]:
assert len(train_src_clean) == len(train_tgt_clean)

In [12]:
directory = '/home/ml/cadencao/XSum/clean_files/'

with open(directory + 'train.source', 'w', encoding='utf-8') as sf:
    with open(directory + 'train.target', 'w', encoding='utf-8') as tf:
        for s, t in zip(train_src_clean, train_tgt_clean):
            sf.write(s + '\n')
            tf.write(t + '\n')